In [ ]:
# for domonkos to run on COLAB

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/python/Computer_Vision/TDT4265_Project/')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.data import load_train_test_val, load_tee
from src.visualize import plot_loss_acc
from src.model import Unet2D
#from src.baseline_model import Unet2D
from src.train import train
from src.utils import save_result, to_cuda
from src.test import test
from src.metrics import acc_metric

from datetime import datetime

from params import *

import torch
from torch import nn

## Prepare data

In [ ]:
train_data, test_data, valid_data = load_train_test_val(DATA_PARAMS, PREP_STEPS, TRAIN_TRANSFORMS)

## Train

In [ ]:
# MODEL: Unet2D (one input channel, 4 output channels)
unet = Unet2D(1, out_channels=4)
print(unet)

opt = torch.optim.Adam(unet.parameters(), lr=LEARN_RATE)

In [ ]:
if LOAD: # Load pretrained model
    unet.load_state_dict(torch.load(MODEL_PATH+FILE_NAME))
    to_cuda(unet)
    print(f"The pre-trained model'{FILE_NAME}' has been loaded")
else:  # Start the training process
    start = datetime.now()
    train_loss, valid_loss, train_acc, valid_acc = train(unet, train_data, valid_data, LOSS, opt, acc_metric, epochs=EPOCHS)

    end = datetime.now()
    print(f"Elapsed time is {str(end-start)}")

In [ ]:
if not LOAD:
    plot_loss_acc(train_loss, valid_loss, train_acc, valid_acc)

## Test

In [ ]:
print("TEST on the CAMUS dataset")
accuracy, average_dice, class_dice = test(unet, test_data, True)

In [ ]:
print(f"Test on TEE images")
tee_data = load_tee(DATA_PARAMS['base_path'], DATA_PARAMS['batch_size'])
accuracy_tee, average_dice_tee, class_dice_tee = test(unet, tee_data, True)

# Save model

In [ ]:
file_name = 'Gauss_bringth_scale15_model'

# Save the model
save_result(unet, MODEL_PATH, file_name, accuracy_tee, average_dice_tee, class_dice_tee)